<a href="https://colab.research.google.com/github/pranshumalik14/ece421-labs-hw/blob/main/labs/lab2/lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 2: Neural Networks

In this lab, we will be creating a .

## 1. Neural Networks using Numpy

We implement.


### 1.1 Helper Functions, Network Definitions, and Initialization

In [56]:
import numpy as np
import matplotlib.pyplot as plt

We first start with defining the activation functions along with their derivatives, which we will later be able to use in our neural networks.

In [57]:
# activation functions
class ReLU:
    # relu = max(x, 0)    
    @staticmethod
    def __call__(x):
        return np.maximum(x, 0)
    # derivative of relu
    @staticmethod
    def prime(x):
        return np.where(x > 0.0, 1.0, 0.0)

class Softmax:
    # softmax = exp(xi)/sum(exp(xj)), applied row-wise to match data
    @staticmethod
    def __call__(x):
        # subtract maximum for numerical stability
        # while keeping the relative differences intact
        x -= x.max(axis=1, keepdims=True)
        return np.exp(x)/np.exp(x).sum(axis=1, keepdims=True)

    @staticmethod
    def prime(x):
        raise NotImplementedError # todo: understand how this will be used

Now, we define the `Network` class that will be used to define an arbitrary network with dense (fully-connected) layers. The class definition and functionality will expand as we progress through each section. We begin with defining the functions that create the network structure, model parameters, their initialization, and the forward pass through the network.

<TODO: 🚧 theory behind NN structure, notation>

<TODO: 🚧 add info about Xavier initialization>

In [58]:
class Network:
    def __init__(self, layer_sizes, layer_activations):
        assert len(layer_activations) == len(layer_sizes)-1
        self.sizes   = layer_sizes
        self.thetas  = layer_activations

        # xavier init scheme
        n_out = layer_sizes[2:] + [0]
        n_in  = layer_sizes[:-1]
        self.biases  = [np.zeros((1, n)) for n in layer_sizes[1:]] # row-vecs
        self.weights = [np.random.randn(m,n)/np.sqrt((n_out[i]+n_in[i])/2) for 
                        i, (m, n) in enumerate(zip(layer_sizes[:-1], layer_sizes[1:]))]
    
    def fwd_prop(self, X):
        X_ell = X
        for W, b, theta in zip(self.weights, self.biases, self.thetas):
            X_ell = theta((X_ell @ W) + b)
        return X_ell

We will now define the loss function we will use to train our model, along with the accuracy model <>... 🚧🚧🚧🚧. Like lab 1, we will use the cross entropy loss since it maximum likelihood and mutually exlusive outcomes and is fit for the classification task at hand.

In [59]:
# loss function
class CrossEntropy:
    @staticmethod
    def __call__(p_hat, ys):
        return -np.sum(ys*np.log(p_hat))

    @staticmethod
    def derivative(logits, target):
        raise NotImplementedError # todo: check inputs and requirement

We also define the `accuracy` function to compute the mean number of misclassifications, giving us an idea of how likely it is for the network to arrive at an incorrect label.

In [68]:
# accuracy of prediction
def accuracy(p_hat, ys):
    return np.mean(np.argmax(ys, axis=1) == np.argmax(p_hat, axis=1))

We will run a forward pass on an untrained, newly initialized, network and test for its accuracy to make sure our implementation is functional thus far. We source the data from section 1.3 just for this purpose.

In [72]:
# create neural network
F, H, K = 784, 1000, 10
nn = Network([F, H, K], [ReLU(), Softmax()])

# forward pass, loss, and accuracy on random init
p_hat    = nn.fwd_prop(X_train)
accur    = accuracy(p_hat, ys_train)
avg_loss = CrossEntropy()(p_hat, ys_train)/X_train.shape[0]
print("Loss = {}, Accuracy = {}".format(avg_loss, accur))

Loss = 2.741738154007161, Accuracy = 0.1128


### 1.2 Backpropagation Derivation

We derive the gradient of the loss function with respect to the model parameters, to be able to update them.

### 1.3 Learning

Training and data

In [62]:
# data loader and associated helper functions
def loadDataGDrive():
    with np.load('/content/drive/MyDrive/Colab Notebooks/notMNIST.npz') as data:
        data, targets = data["images"], data["labels"]
        
        np.random.seed(521)
        rand_idx = np.arange(len(data))
        np.random.shuffle(rand_idx)
        
        data = data[rand_idx] / 255.0
        targets = targets[rand_idx].astype(int)
        
        train_data, train_target = data[:10000], targets[:10000]
        valid_data, valid_target = data[10000:16000], targets[10000:16000]
        test_data, test_target = data[16000:], targets[16000:]
        train_target, valid_target, test_target = convert_onehot(train_target, 
                                                                 valid_target, 
                                                                 test_target)
    return train_data, valid_data, test_data, train_target, valid_target, test_target

def convert_onehot(train_target, valid_target, test_target):
    new_train = np.zeros((train_target.shape[0], 10))
    new_valid = np.zeros((valid_target.shape[0], 10))
    new_test = np.zeros((test_target.shape[0], 10))

    for item in range(0, train_target.shape[0]):
        new_train[item][train_target[item]] = 1
    for item in range(0, valid_target.shape[0]):
        new_valid[item][valid_target[item]] = 1
    for item in range(0, test_target.shape[0]):
        new_test[item][test_target[item]] = 1
    return new_train, new_valid, new_test

def shuffle(data, target):
    np.random.seed(421)
    rand_idx = np.random.permutation(len(data))
    return data[rand_idx], target[rand_idx]

# load data
x_train, x_valid, x_test, ys_train, ys_valid, ys_test = loadDataGDrive()

In [63]:
# flatten datavectors
X_train = x_train.reshape(x_train.shape[0], -1)
X_valid = x_valid.reshape(x_valid.shape[0], -1)
X_test  = x_test.reshape(x_test.shape[0], -1)

# data for training and testing our neural network
Xs = np.array([X_train, X_valid, X_test], dtype=object)    # training, validation, and testing data
Ys = np.array([ys_train, ys_valid, ys_test], dtype=object) # training, validation, and testing labels

In [64]:
def plot_learning_history(title, loss_hist, acc_hist):
    # loss_hist = [train_loss, valid_loss, test_loss]
    # acc_hist  = [train_acc, valid_acc, test_acc]
    f, ax  = plt.subplots(1, 2, figsize=(8,4))
    labels = ["Train", "Validation", "Test"]

    for i in range(loss_hist.shape[1]):
        ax[0].plot(loss_hist[:, i], label=labels[i]) # plot loss history on left
        ax[1].plot(acc_hist[:, i], label=labels[i])  # plot accuracy history on right
    
    # add lables and title
    ax[0].set_xlabel("Epochs")
    ax[0].set_ylabel("Loss")
    ax[0].legend()
    ax[1].set_xlabel("Epochs")
    ax[1].set_ylabel("Accuracy")
    ax[1].legend()
    f.suptitle(title, fontsize=14, y=1.03)
    return plt.tight_layout()